MNIST Dataset 사용하기  
	- 손으로 적혀진 숫자 데이터셋  
	- 28x28=784 픽셀에 숫자가 적혀짐  
출력은 0~9이므로, 10개의 원핫 인코딩을 사용  

Tensorflow에서 제공하는 mnist 라이브러리 사용  
전부는 필요하지 않으므로, batch 함수를 이용해서 100개씩 읽어들일 수 있다.  

#버전 문제 해결  
https://stackoverflow.com/questions/65791987/tensorflow-datasets-version-compatible-with-tensorflow-1-15

Tensorflow Datasets 3.2.1버전과 TensorFlow 1.15.x버전이 호환됨  
CMD창에  
pip install tensorflow==1.15.5  
pip install tensorflow_datasets==3.2.1  
입력해서 설치  

In [18]:
# 설치상태 확인
print(tf.__version__)
print(tfds.__version__)

1.15.5
3.2.1


In [25]:

# Lab 7 Learning rate and Evaluation
import tensorflow as tf
import matplotlib.pyplot as plt
import random

tf.set_random_seed(777)  # for reproducibility

#from tensorflow.examples.tutorials.mnist import input_data #1.15에서 없어짐

# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset
#mnist = input_data.read_data_sets("MNIST_data/", one_hot=True) #원핫 옵션을 통해 읽어오는 순간부터 원핫으로 읽어올 수 있다.  


#버전문제 해결-시도 중-미해결
import tensorflow_datasets as tfds 
#mnist=tfds.image_classification.MNIST

tensorflow_datasets.image_classification.mnist.MNIST

In [ ]:
nb_classes = 10

# MNIST data image of shape 28 * 28 = 784
X = tf.placeholder(tf.float32, [None, 784])
# 0 - 9 digits recognition = 10 classes
Y = tf.placeholder(tf.float32, [None, nb_classes])

#입력이 784개, 출력이 10개이므로 W의 Shape는 784, 10으로 결정됨
W = tf.Variable(tf.random_normal([784, nb_classes]))
b = tf.Variable(tf.random_normal([nb_classes]))

# Hypothesis (using softmax)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

#이번에는 cross entropy함수의 정의를 이용해서 cost함수 구현
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Test model #원핫인코딩된 값을 argmax()로 처리하여 답을 찾는다.
is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [ ]:
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    # Training cycle
    for epoch in range(num_epochs):
        avg_cost = 0

        for i in range(num_iterations):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, cost_val = sess.run([train, cost], feed_dict={X: batch_xs, Y: batch_ys})
            avg_cost += cost_val / num_iterations

        print("Epoch: {:04d}, Cost: {:.9f}".format(epoch + 1, avg_cost))

    print("Learning finished")

epoch: a period of time that is very important in history  
전체 데이터를 다 학습시키면 one epoch이라고 말함.  
데이터가 너무 크면 batch size를 정의해서 잘라냄.
데이터가 1000개이고 batch size가 500일 경우 one epoch를 위해서는 2번 반복(iteration)하면 됨.

epoch이 15이면 전체 데이터셋을 15번을 반복한다는 것  

In [ ]:
    # Test the model using test sets
    print(
        "Accuracy: ",
        accuracy.eval(
            session=sess, feed_dict={X: mnist.test.images, Y: mnist.test.labels}
        ),
    )
    #eval() 함수를 호출하는 것과 sess.run()을 통해 정확도를 계산하는 것은 결과가 같다.

    #matplotlib과 random 모듈을 이용해서 무작위로 숫자를 하나 고르고, 잘 실행되고 있는지 보여준다.
    # Get one and predict
    r = random.randint(0, mnist.test.num_examples - 1)
    print("Label: ", sess.run(tf.argmax(mnist.test.labels[r : r + 1], 1)))
    print(
        "Prediction: ",
        sess.run(tf.argmax(hypothesis, 1), feed_dict={X: mnist.test.images[r : r + 1]}),
    )

    plt.imshow(
        mnist.test.images[r : r + 1].reshape(28, 28),
        cmap="Greys",
        interpolation="nearest",
    )
    plt.show()

'''
Epoch: 0001, Cost: 2.826302672
Epoch: 0002, Cost: 1.061668952
Epoch: 0003, Cost: 0.838061315
Epoch: 0004, Cost: 0.733232745
Epoch: 0005, Cost: 0.669279885
Epoch: 0006, Cost: 0.624611836
Epoch: 0007, Cost: 0.591160344
Epoch: 0008, Cost: 0.563868987
Epoch: 0009, Cost: 0.541745171
Epoch: 0010, Cost: 0.522673578
Epoch: 0011, Cost: 0.506782325
Epoch: 0012, Cost: 0.492447643
Epoch: 0013, Cost: 0.479955837
Epoch: 0014, Cost: 0.468893674
Epoch: 0015, Cost: 0.458703488
Learning finished
Accuracy:  0.8951
'''